# 📈 Pronóstico de Ventas con Prophet

## Objetivo
Predecir los ingresos totales de los próximos 90 días usando **Prophet**, una librería de Meta diseñada para hacer pronósticos de series de tiempo de alta calidad.

### ¿Qué es Prophet?
Prophet es un algoritmo de pronóstico desarrollado por Meta (Facebook) que:
- Detecta automáticamente **tendencias** (crecimiento o decrecimiento)
- Identifica **estacionalidad semanal** (patrones dentro de la semana)
- Identifica **estacionalidad anual** (patrones a lo largo del año)
- Maneja **días festivos** automáticamente
- Genera **intervalos de confianza** (predicciones con rango)

### Librerías Principales
```python
pandas       → Manipulación de datos y agregaciones temporales
prophet      → Modelado de series de tiempo y pronósticos
matplotlib   → Visualización de resultados
numpy        → Operaciones numéricas
```

## Paso 1: Importar Librerías y Configurar Entorno

Importaremos todas las herramientas necesarias para el análisis de series de tiempo.

In [ ]:
# ============================================================================
# PASO 1: IMPORTACIONES Y CONFIGURACIÓN
# ============================================================================

import pandas as pd                          # Manipulación de datos
import numpy as np                           # Operaciones numéricas
import matplotlib.pyplot as plt              # Visualización
import seaborn as sns                        # Gráficos avanzados
from prophet import Prophet                  # Time series forecasting
import warnings                              # Control de advertencias

# Silenciar advertencias de Prophet (es normal que las genere)
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Configurar estilo de gráficos
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print("✅ Todas las librerías importadas correctamente")
print(f"Prophet versión: {Prophet.__module__}")

## Paso 2: Cargar y Explorar Datos Históricos

Cargaremos los datos del Proyecto 1 y realizaremos limpieza básica.

In [ ]:
# ============================================================================
# PASO 2: CARGAR Y EXPLORAR DATOS
# ============================================================================

# Cargar datos del Proyecto 1
ruta_datos = '../project1/data.csv'
df = pd.read_csv(ruta_datos, encoding='latin1')

# Exploración inicial
print("📊 EXPLORACIÓN INICIAL DE DATOS")
print(f"Forma del dataset: {df.shape}")
print(f"\nColumnas disponibles:")
print(df.columns.tolist())
print(f"\nPrimeras 5 filas:")
print(df.head())
print(f"\nTipos de datos:")
print(df.dtypes)
print(f"\nValores faltantes:")
print(df.isnull().sum())

## Paso 3: Limpieza de Datos

Aplicamos filtros para garantizar datos de calidad.

In [ ]:
# ============================================================================
# PASO 3: LIMPIEZA DE DATOS
# ============================================================================

print("🧹 LIMPIEZA DE DATOS")
print(f"Registros iniciales: {len(df):,}")

# 1. Eliminar valores faltantes en CustomerID
df = df.dropna(subset=['CustomerID'])
print(f"Después de eliminar CustomerID faltantes: {len(df):,}")

# 2. Convertir InvoiceDate a datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# 3. Validar que Quantity y UnitPrice sean positivos
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]
print(f"Después de validar Quantity y UnitPrice: {len(df):,}")

# 4. Crear columna TotalPrice
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# 5. Verificar rango de fechas
fecha_min = df['InvoiceDate'].min()
fecha_max = df['InvoiceDate'].max()
print(f"\nRango de fechas: {fecha_min.date()} a {fecha_max.date()}")
print(f"Total de días en dataset: {(fecha_max - fecha_min).days} días")

# 6. Resumen final
print(f"\n✅ Limpieza completada")
print(f"Registros finales para análisis: {len(df):,}")
print(f"Ingresos totales históricos: ${df['TotalPrice'].sum():,.2f}")

## Paso 4: Agregar Datos por Día (Resample)

**¿Por qué agregamos por día?**

Prophet trabaja mejor con series de tiempo regular (una observación por período de tiempo). Agrupamos todas las transacciones de un día en una sola fila con el total de ingresos de ese día.

**¿Qué es `resample`?**

`resample` es una función de Pandas que nos permite agrupar datos por intervalos de tiempo:
- `'D'` = Diario (Daily)
- `'W'` = Semanal (Weekly)
- `'M'` = Mensual (Monthly)

In [ ]:
# ============================================================================
# PASO 4: AGREGAR DATOS POR DÍA (RESAMPLE)
# ============================================================================

print("📅 AGREGACIÓN DIARIA DE VENTAS")

# Establecer InvoiceDate como índice para usar resample
df_temp = df.set_index('InvoiceDate')

# Resample: agrupar por día y sumar TotalPrice
# 'D' = Diario, sum() = sumar todos los ingresos del día
df_daily = df_temp['TotalPrice'].resample('D').sum().reset_index()
df_daily.columns = ['InvoiceDate', 'TotalPrice']

print(f"Período diario: {df_daily['InvoiceDate'].min().date()} a {df_daily['InvoiceDate'].max().date()}")
print(f"Total de días (con y sin ventas): {len(df_daily)}")
print(f"Días con ventas: {(df_daily['TotalPrice'] > 0).sum()}")
print(f"Días sin ventas: {(df_daily['TotalPrice'] == 0).sum()}")

# Estadísticas diarias
print(f"\n💰 ESTADÍSTICAS DIARIAS")
print(f"Ingreso promedio por día: ${df_daily['TotalPrice'].mean():,.2f}")
print(f"Ingreso máximo en un día: ${df_daily['TotalPrice'].max():,.2f}")
print(f"Ingreso mínimo en un día: ${df_daily['TotalPrice'].min():,.2f}")
print(f"Desviación estándar: ${df_daily['TotalPrice'].std():,.2f}")

# Mostrar muestra de datos
print(f"\n📊 Muestra de datos diarios:")
print(df_daily.head(10))
print(f"\n...\n")
print(df_daily.tail(10))

## Paso 5: Formatear Datos para Prophet

Prophet requiere exactamente dos columnas:
- `ds`: Fecha (datestamp)
- `y`: Valor a predecir

In [ ]:
# ============================================================================
# PASO 5: FORMATEAR DATOS PARA PROPHET
# ============================================================================

print("🔧 FORMATEO DE DATOS PARA PROPHET")

# Prophet requiere columnas específicas: 'ds' (fecha) y 'y' (valor)
df_prophet = df_daily.rename(columns={
    'InvoiceDate': 'ds',
    'TotalPrice': 'y'
})

# Verificar que no hay valores nulos (Prophet los rechaza)
print(f"Valores nulos en 'ds': {df_prophet['ds'].isnull().sum()}")
print(f"Valores nulos en 'y': {df_prophet['y'].isnull().sum()}")

# Información final
print(f"\n✅ DataFrame listo para Prophet")
print(f"Forma: {df_prophet.shape}")
print(f"Columnas: {df_prophet.columns.tolist()}")
print(f"\nPrimeras 5 filas:")
print(df_prophet.head())
print(f"\nÚltimas 5 filas:")
print(df_prophet.tail())

## Paso 6: Entrenar el Modelo Prophet

Aquí ocurre la magia. Prophet detectará automáticamente:
- **Tendencia (Trend)**: ¿Crecimiento o decrecimiento general?
- **Estacionalidad Semanal (Weekly)**: Patrones dentro de la semana
- **Estacionalidad Anual (Yearly)**: Patrones a lo largo del año

In [ ]:
# ============================================================================
# PASO 6: ENTRENAR MODELO PROPHET
# ============================================================================

print("🤖 ENTRENAMIENTO DE MODELO PROPHET")
print("\nEsto puede tomar 30-60 segundos (es normal)...\n")

# Inicializar modelo Prophet
# seasonality_mode='multiplicative' = la estacionalidad aumenta con la tendencia
# yearly_seasonality=True = detectar patrones anuales
# weekly_seasonality=True = detectar patrones semanales
# daily_seasonality=False = no hay suficientes datos para diarios
m = Prophet(
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    interval_width=0.95  # Intervalo de confianza al 95%
)

# Entrenar el modelo
print("⏳ Ajustando modelo...")
m.fit(df_prophet)

print("✅ ¡Modelo entrenado correctamente!")
print(f"\nDetalles del modelo:")
print(f"- Modo de estacionalidad: Multiplicativo")
print(f"- Estacionalidad anual: Detectada")
print(f"- Estacionalidad semanal: Detectada")
print(f"- Intervalo de confianza: 95%")

## Paso 7: Crear Pronóstico para los Próximos 90 Días

Le pediremos al modelo que prediga los ingresos diarios para los próximos 90 días.

In [ ]:
# ============================================================================
# PASO 7: CREAR PRONÓSTICO PARA 90 DÍAS
# ============================================================================

print("🔮 CREANDO PRONÓSTICO PARA LOS PRÓXIMOS 90 DÍAS")

# Crear DataFrame "futuro" con 90 días adicionales
# periods=90 significa 90 días hacia adelante
future = m.make_future_dataframe(periods=90, freq='D')

print(f"Rango del DataFrame futuro:")
print(f"- Inicio: {future['ds'].min().date()}")
print(f"- Fin: {future['ds'].max().date()}")
print(f"- Total de filas: {len(future)}")
print(f"\nÚltimas fechas incluidas en el pronóstico:")
print(future.tail(10))

# Generar predicciones
print("\n⏳ Generando predicciones...")
forecast = m.predict(future)

print("✅ Pronóstico completado")
print(f"\nColumnas generadas por Prophet:")
print(f"- ds: Fecha")
print(f"- yhat: Predicción puntual (mejor estimación)")
print(f"- yhat_lower: Límite inferior del intervalo (peor escenario)")
print(f"- yhat_upper: Límite superior del intervalo (mejor escenario)")
print(f"- trend: Componente de tendencia")

## Paso 8: Analizar Pronóstico

Examinamos el pronóstico en detalle.

In [ ]:
# ============================================================================
# PASO 8: ANALIZAR PRONÓSTICO
# ============================================================================

print("📊 ANÁLISIS DEL PRONÓSTICO")

# Separar pronóstico en dos partes: histórico y futuro
forecast_history = forecast[forecast['ds'] <= df_prophet['ds'].max()]
forecast_future = forecast[forecast['ds'] > df_prophet['ds'].max()]

print(f"\n📈 PRONÓSTICO HISTÓRICO (Validación del modelo)")
print(f"Período: {forecast_history['ds'].min().date()} a {forecast_history['ds'].max().date()}")
print(f"Registros: {len(forecast_history)}")
print(f"\nPrimeras 5 predicciones (comparadas con datos reales):")

# Comparar con datos reales
comparison = forecast_history[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head(5)
actual = df_prophet[['ds', 'y']].head(5)
comparison = comparison.merge(actual, on='ds', how='left')
print(comparison.to_string(index=False))

print(f"\n\n🔮 PRONÓSTICO FUTURO (Próximos 90 días)")
print(f"Período: {forecast_future['ds'].min().date()} a {forecast_future['ds'].max().date()}")
print(f"Registros: {len(forecast_future)}")

print(f"\nPronóstico para los próximos 90 días:")
display_future = forecast_future[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
display_future.columns = ['Fecha', 'Predicción', 'Límite Inferior (95%)', 'Límite Superior (95%)']
print(display_future.to_string(index=False))

# Cálculos de resumen
print(f"\n\n💰 ESTIMACIONES FINANCIERAS PARA LOS PRÓXIMOS 90 DÍAS")
ingresos_predichos = forecast_future['yhat'].sum()
ingresos_min = forecast_future['yhat_lower'].sum()
ingresos_max = forecast_future['yhat_upper'].sum()

promedio_diario_predicho = forecast_future['yhat'].mean()
promedio_diario_historico = df_daily['TotalPrice'].mean()
cambio_porcentaje = ((promedio_diario_predicho - promedio_diario_historico) / promedio_diario_historico) * 100

print(f"\nIngresos predichos (90 días):")
print(f"  Predicción puntual:        ${ingresos_predichos:,.2f}")
print(f"  Límite inferior (95%):     ${ingresos_min:,.2f}")
print(f"  Límite superior (95%):     ${ingresos_max:,.2f}")
print(f"  Rango de incertidumbre:    ${ingresos_max - ingresos_min:,.2f}")

print(f"\nPromedio diario:")
print(f"  Histórico: ${promedio_diario_historico:,.2f}")
print(f"  Predicción: ${promedio_diario_predicho:,.2f}")
print(f"  Cambio: {cambio_porcentaje:+.2f}%")

# Desglose trimestral (30 días cada período)
print(f"\n📅 DESGLOSE POR TRIMESTRE (30 días cada uno)")
for i in range(3):
    inicio = i * 30
    fin = (i + 1) * 30
    trimestre = forecast_future.iloc[inicio:fin]
    ingresos_trim = trimestre['yhat'].sum()
    promedio_trim = trimestre['yhat'].mean()
    print(f"\nTrimestre {i+1} (días {inicio+1}-{fin}):")
    print(f"  Ingresos totales: ${ingresos_trim:,.2f}")
    print(f"  Promedio diario: ${promedio_trim:,.2f}")

## Paso 9: Visualizar Pronóstico General

Gráfico principal que muestra:
- Puntos negros: Datos históricos reales
- Línea azul: Pronóstico
- Área azul clara: Intervalo de confianza (95%)

In [ ]:
# ============================================================================
# PASO 9: VISUALIZAR PRONÓSTICO GENERAL
# ============================================================================

print("📊 VISUALIZANDO PRONÓSTICO GENERAL")

# Crear gráfico principal
fig = m.plot(forecast, figsize=(16, 7))

# Personalizar el gráfico
plt.title('Pronóstico de Ventas: Histórico + 90 Días Futuros', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Fecha', fontsize=12, fontweight='bold')
plt.ylabel('Ingresos Totales Diarios (USD)', fontsize=12, fontweight='bold')
plt.grid(True, alpha=0.3)

# Agregar leyenda
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='black', label='Datos Históricos (Reales)'),
    Patch(facecolor='blue', label='Pronóstico'),
    Patch(facecolor='lightblue', label='Intervalo de Confianza (95%)')
]
plt.legend(handles=legend_elements, loc='upper left', fontsize=11)

# Destacar período de pronóstico con una línea vertical
fecha_inicio_pronos = df_prophet['ds'].max()
plt.axvline(x=fecha_inicio_pronos, color='red', linestyle='--', linewidth=2, label='Inicio del Pronóstico')

plt.tight_layout()
plt.savefig('pronóstico_general.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Gráfico guardado como 'pronóstico_general.png'")

## Paso 10: Visualizar Componentes (LA PARTE MÁS IMPORTANTE)

Los gráficos de componentes nos dicen **POR QUÉ** el modelo hace sus predicciones.

Analiza:
1. **Trend**: ¿La empresa crece o decrece?
2. **Weekly**: ¿Qué día de la semana se vende más?
3. **Yearly**: ¿Qué mes del año tiene más ventas?

In [ ]:
# ============================================================================
# PASO 10: VISUALIZAR COMPONENTES DEL MODELO
# ============================================================================

print("🔍 VISUALIZANDO COMPONENTES DEL MODELO")
print("\nEsto nos muestra por qué el modelo hace sus predicciones:")
print("1. TREND: Tendencia general a lo largo del tiempo")
print("2. YEARLY: Patrones estacionales anuales")
print("3. WEEKLY: Patrones estacionales semanales")

# Graficar componentes
fig = m.plot_components(forecast, figsize=(16, 10))

# Ajustar el título general
fig.suptitle('Componentes del Pronóstico: Análisis de Patrones', fontsize=16, fontweight='bold', y=0.995)

plt.tight_layout()
plt.savefig('componentes_pronóstico.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Gráfico de componentes guardado como 'componentes_pronóstico.png'")

## Paso 11: Interpretar Componentes

Análisis en profundidad de cada componente del modelo.

In [ ]:
# ============================================================================
# PASO 11: INTERPRETAR COMPONENTES
# ============================================================================

print("\n" + "="*80)
print("INTERPRETACIÓN DETALLADA DE COMPONENTES")
print("="*80)

# Obtener componentes individuales
components = forecast[['ds', 'trend', 'yearly', 'weekly']].copy()
components_historical = components[components['ds'] <= df_prophet['ds'].max()]

# 1. ANÁLISIS DE TENDENCIA (TREND)
print("\n📈 1. ANÁLISIS DE TENDENCIA (TREND)")
print("-" * 80)
trend_inicio = components_historical['trend'].iloc[0]
trend_fin = components_historical['trend'].iloc[-1]
cambio_trend = trend_fin - trend_inicio
pct_trend = (cambio_trend / trend_inicio) * 100 if trend_inicio != 0 else 0

print(f"Valor inicial de tendencia: ${trend_inicio:,.2f}")
print(f"Valor final de tendencia: ${trend_fin:,.2f}")
print(f"Cambio absoluto: ${cambio_trend:,.2f}")
print(f"Cambio porcentual: {pct_trend:+.2f}%")

if pct_trend > 0:
    print(f"\n✅ INTERPRETACIÓN: La empresa está en CRECIMIENTO")
    print(f"   Las ventas muestran una tendencia positiva a lo largo del tiempo.")
else:
    print(f"\n⚠️ INTERPRETACIÓN: La empresa está en DECRECIMIENTO")
    print(f"   Las ventas muestran una tendencia negativa a lo largo del tiempo.")

# 2. ANÁLISIS DE ESTACIONALIDAD SEMANAL
print("\n\n📅 2. ANÁLISIS DE ESTACIONALIDAD SEMANAL (WEEKLY)")
print("-" * 80)

# Extraer el impacto semanal
weekly_data = forecast[['ds', 'weekly']].copy()
weekly_data['day_of_week'] = weekly_data['ds'].dt.day_name()
weekly_avg = weekly_data.groupby('day_of_week')['weekly'].mean()

# Orden correcto de días
dias_orden = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dias_español = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']

print("\nImpacto de cada día de la semana en las ventas:")
print("(Valores positivos = venta SUPERIOR al promedio)")
print("(Valores negativos = venta INFERIOR al promedio)\n")

for dia_ing, dia_esp in zip(dias_orden, dias_español):
    if dia_ing in weekly_avg.index:
        valor = weekly_avg[dia_ing]
        porcentaje = (valor / df_daily['TotalPrice'].mean()) * 100
        barra = '█' * max(0, int(abs(valor) / 10))
        print(f"{dia_esp:12} {valor:>8.2f}  ({porcentaje:>+6.1f}%)  {barra}")

# Identificar mejor y peor día
mejor_dia_idx = weekly_avg.idxmax()
peor_dia_idx = weekly_avg.idxmin()
mejor_dia_esp = dias_español[dias_orden.index(mejor_dia_idx)]
peor_dia_esp = dias_español[dias_orden.index(peor_dia_idx)]

print(f"\n✅ MEJOR DÍA: {mejor_dia_esp} (venta {weekly_avg[mejor_dia_idx]:+.2f} sobre promedio)")
print(f"⚠️ PEOR DÍA: {peor_dia_esp} (venta {weekly_avg[peor_dia_idx]:+.2f} sobre promedio)")

diferencia_dia = weekly_avg[mejor_dia_idx] - weekly_avg[peor_dia_idx]
print(f"\n💡 Diferencia entre mejor y peor día: {diferencia_dia:.2f} ({(diferencia_dia/df_daily['TotalPrice'].mean())*100:.1f}%)")

# 3. ANÁLISIS DE ESTACIONALIDAD ANUAL
print("\n\n📆 3. ANÁLISIS DE ESTACIONALIDAD ANUAL (YEARLY)")
print("-" * 80)

# Extraer el impacto anual
yearly_data = forecast[['ds', 'yearly']].copy()
yearly_data['month'] = yearly_data['ds'].dt.month
yearly_avg = yearly_data.groupby('month')['yearly'].mean()

meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 
         'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

print("\nImpacto de cada mes del año en las ventas:")
print("(Valores positivos = mes con MAYOR venta)")
print("(Valores negativos = mes con MENOR venta)\n")

for mes_num, mes_nombre in enumerate(meses, 1):
    if mes_num in yearly_avg.index:
        valor = yearly_avg[mes_num]
        porcentaje = (valor / df_daily['TotalPrice'].mean()) * 100
        barra = '█' * max(0, int(abs(valor) / 15))
        print(f"{mes_nombre:12} {valor:>8.2f}  ({porcentaje:>+6.1f}%)  {barra}")

# Identificar mejor y peor mes
mejor_mes_num = yearly_avg.idxmax()
peor_mes_num = yearly_avg.idxmin()
mejor_mes = meses[mejor_mes_num - 1]
peor_mes = meses[peor_mes_num - 1]

print(f"\n✅ MEJOR MES: {mejor_mes} (venta {yearly_avg[mejor_mes_num]:+.2f} sobre promedio)")
print(f"⚠️ PEOR MES: {peor_mes} (venta {yearly_avg[peor_mes_num]:+.2f} sobre promedio)")

diferencia_mes = yearly_avg[mejor_mes_num] - yearly_avg[peor_mes_num]
print(f"\n💡 Diferencia entre mejor y peor mes: {diferencia_mes:.2f} ({(diferencia_mes/df_daily['TotalPrice'].mean())*100:.1f}%)")

## Paso 12: Exportar Resultados

Guardar pronóstico y análisis para referencia futura.

In [ ]:
# ============================================================================
# PASO 12: EXPORTAR RESULTADOS
# ============================================================================

print("\n💾 EXPORTANDO RESULTADOS")

# 1. Exportar pronóstico completo
forecast_export = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'yearly', 'weekly']].copy()
forecast_export.columns = ['Fecha', 'Predicción', 'Límite_Inferior_95%', 'Límite_Superior_95%', 'Tendencia', 'Estacionalidad_Anual', 'Estacionalidad_Semanal']
forecast_export.to_csv('pronóstico_completo.csv', index=False, date_format='%Y-%m-%d')
print("✅ Archivo 'pronóstico_completo.csv' guardado")

# 2. Exportar solo el pronóstico futuro
forecast_future_export = forecast_future[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
forecast_future_export.columns = ['Fecha', 'Predicción', 'Límite_Inferior_95%', 'Límite_Superior_95%']
forecast_future_export.to_csv('pronóstico_90_días.csv', index=False, date_format='%Y-%m-%d')
print("✅ Archivo 'pronóstico_90_días.csv' guardado")

# 3. Guardar el modelo para uso futuro
import pickle
with open('modelo_prophet.pkl', 'wb') as f:
    pickle.dump(m, f)
print("✅ Modelo guardado en 'modelo_prophet.pkl'")

# 4. Crear resumen de insights
resumen = f"""
╔════════════════════════════════════════════════════════════════════╗
║          RESUMEN EJECUTIVO: PRONÓSTICO DE VENTAS 90 DÍAS          ║
╚════════════════════════════════════════════════════════════════════╝

📊 DATOS HISTÓRICOS
─────────────────────────────────────────────────────────────────────
Período cubierto: {df_prophet['ds'].min().date()} a {df_prophet['ds'].max().date()}
Número de días: {len(df_prophet)}
Ingresos totales históricos: ${df_daily['TotalPrice'].sum():,.2f}
Ingreso promedio diario: ${df_daily['TotalPrice'].mean():,.2f}
Ingreso máximo en un día: ${df_daily['TotalPrice'].max():,.2f}
Ingreso mínimo en un día: ${df_daily['TotalPrice'].min():,.2f}

🔮 PRONÓSTICO 90 DÍAS
─────────────────────────────────────────────────────────────────────
Período predicho: {forecast_future['ds'].min().date()} a {forecast_future['ds'].max().date()}
Ingresos predichos (estimación): ${ingresos_predichos:,.2f}
Intervalo de confianza (95%):
  - Mínimo esperado: ${ingresos_min:,.2f}
  - Máximo esperado: ${ingresos_max:,.2f}
Promedio diario predicho: ${promedio_diario_predicho:,.2f}
Cambio vs histórico: {cambio_porcentaje:+.2f}%

📈 ANÁLISIS DE TENDENCIA
─────────────────────────────────────────────────────────────────────
Dirección: {'CRECIMIENTO ✅' if pct_trend > 0 else 'DECRECIMIENTO ⚠️'}
Cambio de tendencia: {pct_trend:+.2f}%
Interpretación: La empresa está en {'expansión' if pct_trend > 0 else 'contracción'}

📅 PATRONES SEMANALES
─────────────────────────────────────────────────────────────────────
Mejor día: {mejor_dia_esp} ({weekly_avg[mejor_dia_idx]:+.2f})
Peor día: {peor_dia_esp} ({weekly_avg[peor_dia_idx]:+.2f})
Diferencia: {diferencia_dia:.2f} ({(diferencia_dia/df_daily['TotalPrice'].mean())*100:.1f}%)

🎄 PATRONES ANUALES
─────────────────────────────────────────────────────────────────────
Mejor mes: {mejor_mes} ({yearly_avg[mejor_mes_num]:+.2f})
Peor mes: {peor_mes} ({yearly_avg[peor_mes_num]:+.2f})
Diferencia: {diferencia_mes:.2f} ({(diferencia_mes/df_daily['TotalPrice'].mean())*100:.1f}%)

💡 RECOMENDACIONES ESTRATÉGICAS
─────────────────────────────────────────────────────────────────────
1. Planificación de Inventario:
   Preparar {promedio_diario_predicho:,.0f} USD en inventario promedio diario

2. Estrategia de Marketing:
   - Enfoque en {mejor_dia_esp}s para maximizar ventas
   - Campañas especiales los {peor_dia_esp}s para aumentar volumen

3. Gestión de Recursos:
   - Incrementar personal en {mejor_mes}
   - Optimizar costos en {peor_mes}

4. Proyección Trimestral:
   - Trimestre Q1: Período 1-30 días
   - Trimestre Q2: Período 31-60 días
   - Trimestre Q3: Período 61-90 días

═══════════════════════════════════════════════════════════════════════
Generado: {pd.Timestamp.now().strftime('%d de %B de %Y - %H:%M:%S')}
═══════════════════════════════════════════════════════════════════════
"""

with open('resumen_pronóstico.txt', 'w', encoding='utf-8') as f:
    f.write(resumen)

print("✅ Resumen guardado en 'resumen_pronóstico.txt'")
print("\n" + resumen)

## Paso 13: Resumen y Próximos Pasos

Conclusiones finales del análisis.

In [ ]:
# ============================================================================
# PASO 13: RESUMEN Y PRÓXIMOS PASOS
# ============================================================================

print("\n" + "="*80)
print("🎉 PROYECTO 5: PRONÓSTICO DE VENTAS CON PROPHET - COMPLETADO")
print("="*80)

print("""
✅ HABILIDADES DEMOSTRADAS:
   • Time Series Analysis (Análisis de Series Temporales)
   • Machine Learning con Prophet
   • Detección de tendencias y estacionalidad
   • Análisis de patrones semanales y anuales
   • Visualización de pronósticos con intervalos de confianza
   • Interpretación de componentes para business intelligence

📊 ARCHIVOS GENERADOS:
   1. pronóstico_general.png              → Gráfico principal del pronóstico
   2. componentes_pronóstico.png          → Análisis de componentes
   3. pronóstico_completo.csv             → Dataset completo con predicciones
   4. pronóstico_90_días.csv              → Solo los 90 días futuros
   5. modelo_prophet.pkl                  → Modelo entrenado para reutilizar
   6. resumen_pronóstico.txt              → Resumen ejecutivo

💼 VALOR PARA EL NEGOCIO:
   ✓ Predicción de ingresos para planificación financiera
   ✓ Identificación de patrones de compra por día y mes
   ✓ Planificación de inventario basada en pronósticos
   ✓ Estrategia de marketing orientada a patrones semanales
   ✓ Preparación para picos de ventas anticipados

🚀 PRÓXIMOS PASOS:
   1. ✅ Ejecutar este notebook y validar resultados
   2. ✅ Crear README.md con explicación de metodología
   3. ✅ Compartir pronóstico con stakeholders
   4. ✅ Monitorear precisión conforme pasen los días
   5. ✅ Reentrenar modelo mensualmente con nuevos datos
   6. ✅ Publicar en LinkedIn como demostración de habilidades

📚 CONCEPTOS TÉCNICOS CUBIERTOS:
   • Resample de Pandas para agregación temporal
   • Algoritmo Prophet y sus parámetros
   • Interpretación de componentes (Trend, Yearly, Weekly)
   • Intervalos de confianza y predicciones con incertidumbre
   • Exportación de modelos para producción

═══════════════════════════════════════════════════════════════════════════════

🎓 Este es el PROYECTO 5 de tu portafolio data science:

   Proyecto 1: Análisis y Visualización (Matplotlib)
   Proyecto 2: Consolidación y Reportes (Pandas + Excel)
   Proyecto 3: Dashboard Interactivo (Streamlit + Plotly)
   Proyecto 4: Segmentación con ML (SQL + K-Means)
   Proyecto 5: Pronóstico de Series Temporales (Prophet) ← TÚ ESTÁS AQUÍ

   ¡Has avanzado desde análisis básicos hasta predicciones de futuro! 🚀

═══════════════════════════════════════════════════════════════════════════════
""")

print("\n✨ ¡PROYECTO 5 COMPLETADO EXITOSAMENTE! ✨\n")